In [3]:
import pandas as pd
import numpy as np

MAX_ADACHI = 6
ADACHI_DAY_LEVELS = [1,2,3,5]

def setup():
  # Data loading and setting up
  df = pd.read_csv('k1.csv')
  availabilities = df[df.columns[2]].astype(str)
  months = df[df.columns[0]].ffill().astype(str)
  date = df[df.columns[1]].astype(np.uint8)
  df = df.drop(df.columns[range(3)], axis=1)
  data = df.to_numpy().astype(np.uint8)

  # SLink mat setup
  sl_max = np.ones((19,)) * 10
  #Adachi
  adachi = df.columns.get_loc('Tohru Adachi')
  sl_max[adachi] = MAX_ADACHI
  #Marie
  sl_max[9] = 4
  #Eri
  eri = df.columns.get_loc('Eri Minami')
  sl_max[eri] = 11
  #Nanako
  suggoi = df.columns.get_loc('Nanako Dojima') #TODO: changeme!
  sl_max[suggoi] = 11 #TODO: changeme!
  mat = np.diag(sl_max).astype(np.uint8)

  listrange = lambda x,y: list(range(x, y))
  day_chars = [*listrange(0, 2), *listrange(3,12), *listrange(14, 17), 18]
  night_chars = sorted(list(set(listrange(0,19))-set(day_chars)))
  return {'availabilities': availabilities, 'mat': mat, 'months': months, 'date': date, 'data': data, 'day_chars': day_chars, 'night_chars': night_chars, 'df': df}


In [6]:
x = setup()
x['df'].columns[9]

'Marie'

## Weights for Kou

In [35]:
# Kou  setup

def setup_kou():
  weights = np.ones((19,))
  return weights

## Weights for Daisuke

In [30]:
# Daisuke setup
def setup_daisuke():
    weights = np.ones((19,))
    return weights

## Simulation

In [31]:
def process_adachi(level, idx, night_chars, day_chars):
    if level in ADACHI_DAY_LEVELS:
        night_chars = night_chars.remove(idx)
        day_chars = day_chars.append(idx)
    else:
        night_chars = night_chars.append(idx)
        day_chars = day_chars.remove(idx)
    return night_chars, day_chars

def iterate(r: np.ndarray, data: np.ndarray, mat: np.ndarray, night_chars, day_chars):
    s = data.sum(axis=0)
    r = np.divide(r, s, out=np.zeros_like(r).astype(np.float32), where=s!=0)
    res = mat@r
    maxdata = np.argmax(res)
    if res[maxdata] <= 0: return mat, None
    curr = mat[maxdata, maxdata]
    if curr <= 0: return mat, None
    mat[maxdata, maxdata] = curr - 1
    adachi = df.columns.get_loc('Tohru Adachi')
    if maxdata == adachi:
        night_chars, day_chars = process_adachi(MAX_ADACHI-curr+1, adachi, night_chars, day_chars)
    return mat, maxdata, night_chars, day_chars

def simulate(data, mat, weights, availabilities, night_chars, day_chars, **kwargs):
    selecteds = []
    for i, r in enumerate(data.copy()):
        r = r * weights
        if i == 105:
            mat[9, 9] = 6 + mat[9, 9]
        if availabilities[i] == 'b':
            a = r.copy()
            a[night_chars] = 0
            d = data[i:].copy()
            d[:, night_chars] = 0
            mat, selected1, night_chars, day_chars = iterate(a, d, mat)
            a = r.copy()
            a[day_chars] = 0
            d = data[i:].copy()
            d[:, day_chars] = 0
            mat, selected2, night_chars, day_chars = iterate(a, d, mat)
            selected = (selected1, selected2)
        else:
            mat, selected, night_chars, day_chars = iterate(r, data[i:], mat)
            selected = (selected, None)
        selecteds.append(selected)
    # print(np.diag(mat))
    # print(df.columns[np.diag(mat) > 0], np.diag(mat)[np.diag(mat) > 0])
    # print(np.where(np.diag(mat) > 0))
    return selecteds, np.diag(mat)


## Find weights

In [39]:
weights = setup_kou()
for i in range(100):
  data = setup()
  data['weights'] = weights
  _, res = simulate(**data)
  if (res == 0).all():
    break
  weights[np.where(res > 0)] += .1
print(f'iteration needed: {i}')
print(f'final sl residue: {res}')
print(f'final sl residue names: {data["df"].columns[res > 0]}')
print(f'weights: {weights}')

  

iteration needed: 50
final sl residue: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
names: Index([], dtype='object')
weights: [1.1 2.1 1.  2.1 1.  1.4 1.4 1.  1.8 1.3 1.  1.  1.  1.  1.1 1.9 1.  1.
 1.5]


## Final simulation and Save result

In [33]:
data = setup()
data['weights'] = weights
df = data['df']
date = data['date']
months = data['months']
selecteds, res = simulate(**data)
print(res)
print(data['df'].columns[res > 0])
names = df.columns.to_numpy().copy()
none = names.shape[0]
names = np.append(names, 'None')
p = []
for i, (d, m) in enumerate(zip(date, months)):
    s1, s2 = selecteds[i]
    if s1 is None: s1 = none
    if s2 is None:
        p.append(names[s1])
    else:
        p.append(f'Day: {names[s1]}, Night: {names[s2]}')

import csv
with open('ok1.csv', 'w') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerows(zip(months,date,p))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Index([], dtype='object')
